# Project

### Import Modules

In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Loading Datasets

In [3]:
FILENAME= "calendar.csv"
DATADIR = "../pfda_project/"

calendar_df = pd.read_csv(DATADIR + FILENAME)

In [4]:
FILENAME= "inventory.csv"
DATADIR = "../pfda_project/"

inventory_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [5]:
FILENAME= "sales_test.csv"
DATADIR = "../pfda_project/"

sales_test_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [6]:
FILENAME= "sales_train.csv"
DATADIR = "../pfda_project/"

sales_train_df = pd.read_csv(DATADIR + FILENAME)

In [7]:
FILENAME= "solution.csv"
DATADIR = "../pfda_project/"

solution_df = pd.read_csv(DATADIR + FILENAME)

In [8]:
FILENAME= "test_weights.csv"
DATADIR = "../pfda_project/"

test_weights_df = pd.read_csv(DATADIR + FILENAME)

In [ ]:
# Merge Data
data = sales_train_df.merge(calendar_df, on='date', how='left').merge(inventory_df, on='unique_id', how='left')

In [13]:
# Date engeneering
data['day_of_week'] = pd.to_datetime(data['date']).dt.dayofweek
data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype(int)
data['is_holiday'] = data['holiday'].fillna(0).astype(int)

In [15]:
# Create lag features
for lag in [1, 7, 30]:
    data[f'sales_lag_{lag}'] = data.groupby('unique_id')['sales'].shift(lag)

In [14]:
# Rolling
data['rolling_mean_7'] = data.groupby('unique_id')['sales'].shift(1).rolling(window=7).mean()

In [16]:
# Drop rows with NaN values after feature engineering
data = data.dropna()

In [22]:
data.head()

,unique_id,date,warehouse_x,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,L3_category_name_en,L4_category_name_en,warehouse,day_of_week,is_weekend,is_holiday,rolling_mean_7,sales_lag_1,sales_lag_7,sales_lag_30
77,4845,2020-12-24,Budapest_1,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,1,28.720000,28.72,28.72,6.39
78,4845,2020-12-24,Budapest_1,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,1,24.617143,0.00,28.72,6.39
79,4845,2020-12-24,Budapest_1,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,1,20.514286,0.00,28.72,12.04
80,4845,2020-12-24,Budapest_1,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,1,16.411429,0.00,28.72,12.04
81,4845,2020-12-24,Budapest_1,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,...,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,1,12.308571,0.00,28.72,12.04


In [23]:
# Train-Test Split
X = data.drop(columns=['sales', 'unique_id', 'date'])
y = data['sales']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)